sarah: make 10,000 the threshold face area that would not encounter change
below code is about the moving forwards and backwards
todo: increase tolerance and threshold.

In [7]:
import cv2
import numpy as np


# Load the cascade for face detection
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

# Start the video capture
cap = cv2.VideoCapture(0)

# Set the initial position of the motor
initial_position = 0

desired_face_area = 0
current_face_area = 0
callibrated = False
movingX = False
movingY = False
text = ""

while True:
    
    # Read the frame
    _, frame = cap.read()
    coordinates = (100,100)
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale=1
    color = (255,0,255)
    thickness = 2

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray,scaleFactor=1.2, minNeighbors=4, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    areas=[]
    if np.any(faces): # faces is not empty
        # find the largest face 
        if (len(faces)>1): # if there are more than one face
            for (x, y, w, h) in faces:
                areas.append((x+w)*(y+h))

            maxArea = max(areas)
            maxAreaPos = areas.index(maxArea)

            face = faces[maxAreaPos] #largest face
        elif (len(faces)==1):
            face = faces[0]
    
        (x,y,w,h) = face 

        # Draw a rectangle around the faces
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Get the center of the face
        face_center_x = x + w/2
        face_center_y = y + h/2
        
        # field of view center
        frame_center_x = frame.shape[1]/2
        frame_center_y = frame.shape[0]/2

        # calculate the area of the desired face rectangle
        current_area = (x+w)*(y+h)
        
        # Calculate the error from the center of the frame
        error_x = frame_center_x - face_center_x
       # error_y = frame_center_y - face_center_y
        
        # left and right motion
        if (error_x>60): 
            text = "turn left" + str(error_x)
            movingX = True
        elif (error_x<-60):
            text = "turn right" + str(error_x)
            movingX = True
        else:
            movingX = False
        
        # callibrate
        if cv2.waitKey(1) & 0xFF == ord('b'):
            desired_face_area = current_area
            callibrated = True
            text = "callibration started"
            
        if callibrated:
            if (current_area - desired_face_area > 100000): #TODO: can change the tolerance
                text = "zoom out"
                movingY = True
            elif (current_area - desired_face_area<-100000):
                text = "zoom in"
                movingY = True
            else:
                movingY = False
        else:
            pass
            #print ("not callibrated")
        
        if not movingX and not movingY:
            text = "stop"+ str(error_x)

    else: #faces empty
        print("faces empty")

    # Show the frame
    frame = cv2.putText(frame, text, coordinates, font, fontScale, color, thickness, cv2.LINE_AA)

    cv2.imshow("Face Tracking", frame)

    # Exit if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture
cap.release()

# Close all windows
cv2.destroyAllWindows()

faces empty
faces empty
faces empty
